# Gallery Example: D/M/1 Queue (Deterministic Arrivals)

This example demonstrates a D/M/1 queueing system:
- **Arrivals**: Deterministic inter-arrival times (constant)
- **Service**: Exponential service times
- **Servers**: 1 server
- **Capacity**: Infinite
- **Scheduling**: FCFS

Deterministic arrivals represent perfectly regular arrival patterns with zero variability (C² = 0).

In [ ]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_detm1():    """Create D/M/1 queueing model"""    model = Network('D/M/1')        # Block 1: nodes    source = Source(model, 'mySource')    queue = Queue(model, 'myQueue', SchedStrategy.FCFS)    sink = Sink(model, 'mySink')        # Block 2: classes    oclass = OpenClass(model, 'myClass')    # Deterministic arrivals with inter-arrival time = 1 (rate = 1)    source.set_arrival(oclass, Det(1.0))         # Exponential service with rate μ=2    queue.set_service(oclass, Exp(2))        # Block 3: topology    P = model.init_routing_matrix()    P.add_route(oclass, source, queue, 1.0)    P.add_route(oclass, queue, sink, 1.0)    model.link(P)        return model# Create the modelmodel = gallery_detm1()print(f"Arrival pattern: Deterministic (λ = 1, C²ₐ = 0)")print(f"Service pattern: Exponential (μ = 2, C²ₛ = 1)")print(f"Utilization: ρ = λ/μ = 1/2 = 0.5")

## Theoretical Analysis for D/M/1

For D/M/1 with:
- **Arrival rate**: λ = 1 (deterministic inter-arrival time = 1)
- **Service rate**: μ = 2 (exponential)
- **Utilization**: ρ = λ/μ = 0.5

Deterministic arrivals characteristics:
- **Inter-arrival time**: Constant = 1.0
- **Variance**: 0 (no variability)
- **Coefficient of Variation**: C²ₐ = 0

D/M/1 typically performs much better than M/M/1 due to zero arrival variability.

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# MVA Solver
solver_mva = MVA(model)
avg_table_mva = solver_mva.avg_table()
print("\nMVA Solver:")
print(avg_table_mva)

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=15)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)

In [ ]:
# Compare D/M/1 with M/M/1 - effect of arrival variability
print("\n=== Impact of Arrival Variability: D/M/1 vs M/M/1 ===")

def create_equivalent_mm1():
    """Create equivalent M/M/1 model"""
    model_mm1 = Network('M/M/1-Equivalent')
    source = Source(model_mm1, 'Source')
    queue = Queue(model_mm1, 'Queue', SchedStrategy.FCFS)
    sink = Sink(model_mm1, 'Sink')
    
    oclass = OpenClass(model_mm1, 'Class')
    source.set_arrival(oclass, Exp(1))  # Same mean arrival rate
    queue.set_service(oclass, Exp(2))   # Same service rate
    
    P = model_mm1.init_routing_matrix()
    P.add_route(oclass, source, queue, 1.0)
    P.add_route(oclass, queue, sink, 1.0)
    model_mm1.link(P)
    
    return model_mm1

model_mm1 = create_equivalent_mm1()
solver_dm1 = MVA(model)
solver_mm1 = MVA(model_mm1)

avg_table_dm1 = solver_dm1.avg_table()
avg_table_mm1 = solver_mm1.avg_table()

print("D/M/1 Results (deterministic arrivals):")
print(avg_table_dm1)

print("\nM/M/1 Results (exponential arrivals):")
print(avg_table_mm1)

# Extract performance metrics for comparison
dm1_resp = float(avg_table_dm1.iloc[1, 2])  # D/M/1 response time
dm1_length = float(avg_table_dm1.iloc[1, 3])  # D/M/1 queue length

mm1_resp = float(avg_table_mm1.iloc[1, 2])  # M/M/1 response time
mm1_length = float(avg_table_mm1.iloc[1, 3])  # M/M/1 queue length

print(f"\nVariability Impact Comparison:")
print(f"Arrival Distribution | C²ₐ | Response Time | Queue Length | Performance")
print("-" * 70)
print(f"Deterministic        |  0  |     {dm1_resp:.4f}    |    {dm1_length:.4f}     | Best")
print(f"Exponential          |  1  |     {mm1_resp:.4f}    |    {mm1_length:.4f}     | Baseline")

improvement = (mm1_resp - dm1_resp) / mm1_resp * 100 if mm1_resp > dm1_resp else 0
print(f"\nDeterministic Advantage: {improvement:.1f}% better response time vs exponential arrivals")
print(f"Queue length reduction: {(mm1_length - dm1_length) / mm1_length * 100:.1f}%")
print("\nKey insight: Eliminating arrival variability significantly improves performance.")

In [ ]:
# Compare with M/D/1 (deterministic service)
print("\n=== D/M/1 vs M/D/1: Arrival vs Service Variability ===")

def create_md1_model():
    """Create M/D/1 model (exponential arrivals, deterministic service)"""
    model_md1 = Network('M/D/1')
    source = Source(model_md1, 'Source')
    queue = Queue(model_md1, 'Queue', SchedStrategy.FCFS)
    sink = Sink(model_md1, 'Sink')
    
    oclass = OpenClass(model_md1, 'Class')
    source.set_arrival(oclass, Exp(1))      # Exponential arrivals
    queue.set_service(oclass, Det(0.5))     # Deterministic service time = 0.5
    
    P = model_md1.init_routing_matrix()
    P.add_route(oclass, source, queue, 1.0)
    P.add_route(oclass, queue, sink, 1.0)
    model_md1.link(P)
    
    return model_md1

def create_dd1_model():
    """Create D/D/1 model (both deterministic)"""
    model_dd1 = Network('D/D/1')
    source = Source(model_dd1, 'Source')
    queue = Queue(model_dd1, 'Queue', SchedStrategy.FCFS)
    sink = Sink(model_dd1, 'Sink')
    
    oclass = OpenClass(model_dd1, 'Class')
    source.set_arrival(oclass, Det(1.0))    # Deterministic arrivals
    queue.set_service(oclass, Det(0.5))     # Deterministic service
    
    P = model_dd1.init_routing_matrix()
    P.add_route(oclass, source, queue, 1.0)
    P.add_route(oclass, queue, sink, 1.0)
    model_dd1.link(P)
    
    return model_dd1

try:
    model_md1 = create_md1_model()
    model_dd1 = create_dd1_model()
    
    solver_md1 = MVA(model_md1)
    solver_dd1 = MVA(model_dd1)
    
    avg_table_md1 = solver_md1.avg_table()
    avg_table_dd1 = solver_dd1.avg_table()
    
    md1_resp = float(avg_table_md1.iloc[1, 2])
    dd1_resp = float(avg_table_dd1.iloc[1, 2])
    
    print("Variability Impact Matrix:")
    print("")
    print("System    | Arrivals C²ₐ | Service C²ₛ | Response Time | Variability Effect")
    print("-" * 75)
    print(f"D/D/1     |      0       |      0      |     {dd1_resp:.4f}    | No variability (best)")
    print(f"D/M/1     |      0       |      1      |     {dm1_resp:.4f}    | Service variability only")
    print(f"M/D/1     |      1       |      0      |     {md1_resp:.4f}    | Arrival variability only")
    print(f"M/M/1     |      1       |      1      |     {mm1_resp:.4f}    | Both variabilities")
    
    print(f"\nVariability Impact Ranking (best to worst):")
    results = [(dd1_resp, 'D/D/1'), (dm1_resp, 'D/M/1'), (md1_resp, 'M/D/1'), (mm1_resp, 'M/M/1')]
    results.sort()
    
    for i, (resp_time, system) in enumerate(results, 1):
        print(f"{i}. {system}: {resp_time:.4f}")
    
    print(f"\nKey Insights:")
    print(f"• Arrival variability impact: {(md1_resp - dd1_resp) / dd1_resp * 100:.1f}% increase")
    print(f"• Service variability impact: {(dm1_resp - dd1_resp) / dd1_resp * 100:.1f}% increase")
    print(f"• Combined effect: {(mm1_resp - dd1_resp) / dd1_resp * 100:.1f}% increase")
    print(f"• Variability in {'arrivals' if md1_resp > dm1_resp else 'service'} has greater impact")

except Exception as e:
    print(f"Detailed comparison error: {e}")
    print("Note: Some solvers may not support deterministic distributions in all configurations.")
    print("The key principle remains: reducing variability improves queueing performance.")